In [108]:
import json
import pandas as pd
from ekonlpy.sentiment import MPCK
from tqdm import tqdm

In [110]:
total_data=pd.read_csv('../preprocess/top10000_review_preprocess.csv', encoding='utf-8')

In [112]:
#total_data=total_data[total_data['star_rating']==2]
reviews=pd.DataFrame(total_data['review_text'].unique())
reviews.columns=['review_text']

In [114]:
reviews['review_text'] = reviews['review_text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
reviews['review_text'].replace('', np.nan, inplace=True)
reviews.dropna(inplace=True)
print(reviews['review_text'].isnull().sum())

0


In [116]:
# Load sentiment word info
with open('SentiWord_info.json', encoding='utf-8-sig', mode='r') as f: 
    SentiWord_info = json.load(f)

sentiword_dic = pd.DataFrame(SentiWord_info)

# Ensure polarity values are integers
sentiword_dic['polarity'] = sentiword_dic['polarity'].astype(int)

# Convert sentiment word dictionary into a lookup dictionary for faster access
sentiment_lookup = dict(zip(sentiword_dic['word'], sentiword_dic['polarity']))


mpck = MPCK()
pos_tag = ['NNG', 'NNP', 'VA', 'MM', 'EP', 'EC', 'ETM', 'XPN']


# Initialize the DataFrame
reviews['tokens_ngrams'] = None
reviews['words_in_sentiword_info'] = None
reviews['sentiment_scores'] = None

# Process each review
for idx, row in tqdm(reviews.iterrows(), total=reviews.shape[0]):
    text = row['review_text']
    tokens = mpck.tokenize(text)
    tokens_list = []
    
    for token in tokens:
        if '/' in token:
            word, tag = token.split('/')
            if tag in pos_tag:
                if tag == 'VA' and word[-1] != '다':
                    word = word + '다'
                tokens_list.append(word)
    
    ngrams = mpck.ngramize(tokens)
    tokens_ngrams = tokens_list + ngrams
    reviews.at[idx, 'tokens_ngrams'] = ','.join(tokens_ngrams)
    
    # Find words in sentiword_info and their scores
    words_in_sentiword_info = [word for word in tokens_ngrams if word in sentiment_lookup]
    sentiment_scores = [sentiment_lookup.get(word, 0) for word in words_in_sentiword_info]
    
    reviews.at[idx, 'words_in_sentiword_info'] = ','.join(words_in_sentiword_info)
    reviews.at[idx, 'sentiment_scores'] = ','.join(map(str, sentiment_scores))

reviews

100%|█████████████████████████████████████████████████████████████████████████████| 3024/3024 [00:09<00:00, 325.88it/s]


,review_text,tokens_ngrams,words_in_sentiword_info,sentiment_scores
0,뭔가 엉망진창이었지만신발자체에는 문제없어서 그냥 구매합니다,"엉망진창,신발,문제없다,구매",,
1,쫌 찝찝한게 있었는데 귀찮아서 기냥 입을라고요 바로 세탁,"찝찝,있다,귀찮다,세탁",귀찮다,-1
2,이거 바로 입을수 없을 정도로 보풀같은게 엄청 묻어있네요배송도 겁나 느렸는데,"없다,보풀,같다,배송,느리다","없다,느리다","-1,-1"
3,가격은 괜찮은데플라스틱인줄 몰랐네요 그리고 배송 포장은 극악,"가격,괜찮다,배송,포장,극악",괜찮다,1
4,알도 다른 제품들 저렴하고 괜찮아서 이번에 이것도 구매했는데 앞부분 모양이 양쪽이 ...,"알,제품,저렴,괜찮다,구매,앞부분,모양,양쪽,달라요다,마감,엉망","괜찮다,모양","1,0"
...,...,...,...,...
3020,진짜 최악 표시보다 훨씬 작아서 한단계 위로 교환받앗는대도 맘에 안듬,"최악,표시,작다,위,교환,맘",최악,-2
3021,생각보다 퀄이 별루지만 이뻐요 ㅎㅎ 할인할때 사서 좋아유,"생각,별루,이뻐요다,할인,때,좋다","할인,좋다","1,2"
3022,음 아주 많이 구겨져와서 스팀다리미와 전기다리미를 동원해도 주름이 펴지지 않아요 혼...,"스팀다리미,전기다리미,주름,혼신,힘,소요,뒷면,주름,불가능,바지,앞면,괜찮다,뒷면,...",괜찮다,1
3023,딱딱한 재질이라서 조금만 넣어도 가방의 쉐입이 망가집니다서류나 노트북처럼 직사각형의...,"딱딱,재질,가방,쉐,입,서류,직사각형,얇다,물건,보관,좋다,두께,조금,물건,보관,불...","좋다,좋다","2,2"
